In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-odia/classification


In [7]:
path = Path('./')

In [8]:
df_train = pd.read_csv(path/'../../indicnlp-news-articles/or/or-train.csv', header=None)
df_train.head()

,0,1
0,sports,Edited By OdishaReporter Bureau | By J G\nOn D...
1,sports,"Last updated Aug 25, 2019\nସେୟାର\nନୂଆଦିଲ୍ଲୀ: ପ..."
2,sports,Edited By OdishaReporter Bureau | By H L\nOn J...
3,entertainment,ରେସ୍ତୁଁରାରେ ଓ୍ଵଟ୍ରେସ ଭାବେ କାମ କରୁଥିଲେ ଏହି ନାୟି...
4,sports,Edited By OdishaReporter Bureau | By R J\nOn N...


In [9]:
df_valid = pd.read_csv(path/'../../indicnlp-news-articles/or/or-valid.csv', header=None)
df_valid.head()

,0,1
0,business,ବିଏସ୍‌ଏନ୍‌ଏଲ୍‌ର ୯୮ ଟଙ୍କିଆ ପ୍ଲାନ୍‌\nବିଏସ୍‌ଏନ୍‌ଏ...
1,entertainment,Edited By OdishaReporter Bureau | By R M\nLast...
2,crime,Edited By OdishaReporter Bureau | By R M\nLast...
3,crime,"ଚାକିରି ଦେବା ନାଁରେ ଠକେଇ, ଅଭିଯୁକ୍ତ ଗିରଫ\nଚାକିରି ..."
4,business,ଭାରତୀୟ ବଜାରରେ ନୂଆ ଟାଟା ଏସିଇ ଗୋଲ୍ଡ\nଭାରତୀୟ ବଜାର...


In [10]:
df_test = pd.read_csv(path/'../../indicnlp-news-articles/or/or-test.csv', header=None)
df_test.head()

,0,1
0,business,0\nସେୟାର\nନୂଆଦିଲ୍ଲୀ: ଗୁଡ୍‌ସ ଆଣ୍ଡ୍‌ ସର୍ଭିସ ଟ୍ୟା...
1,crime,ଫିଲ୍ମ ନିର୍ଦେଶକଙ୍କୁ ଖଣ୍ଡାରେ ଆକ୍ରମଣ\nଫିଲ୍ମ ନିର୍ଦ...
2,entertainment,ତୈମୁରଙ୍କ ପିତା ହେବାକୁ ଚାହାନ୍ତି ଏହି ଅଭିନେତା !\nତ...
3,sports,Edited By OdishaReporter Bureau | By R M\nOn M...
4,entertainment,Edited By OdishaReporter Bureau | By R M\nOn N...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((24000, 2), (3000, 2), (3000, 2))

In [12]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [13]:
label_cols = [0]

In [14]:
class OriyaTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/oriya/tokenizer/oriya_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [15]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/oriya/tokenizer/oriya_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(15000)]

In [16]:
# 15,000 is the vocab size that we chose in sentencepiece
oriya_vocab = Vocab(itos)

In [17]:
tokenizer = Tokenizer(tok_func=OriyaTokenizer, lang='or')

In [18]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [20]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=oriya_vocab)

In [21]:
data_lm.show_batch()

idx,text
0,▁ବରା ବର ▁ସ୍ଥିତି ରେ ▁ରହିଛନ୍ତି । ▁ଭାରତ ▁ଆ ଡେ ଲେ ଡ୍ ▁ଟେଷ୍ଟ ୍ ▁ଜିତି ▁ଥିବା ▁ବେଳେ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ବାଜି ▁ମାରି ଥିଲା । ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ଭାରତ ▁ବିନା ▁ସ୍ପିନ ର ରେ ▁ପଡ଼ିଆ କୁ ▁ଓହ୍ଲାଇ ଥିଲା । ▁ଆହତ ▁ଥିବା ▁କାରଣରୁ ▁ପ୍ରଥମ ▁ଟେଷ୍ଟ ରେ ▁୬ଟି ▁ୱିକେଟ ୍ ▁ନେଇଥିବା ▁ରବି ଚନ୍ଦ୍ର ନ ▁ଅଶ୍ୱ ିନ୍ ▁ ଙ୍କୁ ▁ଏହି ▁ଟେଷ୍ଟ ରେ ▁ସାମିଲ ▁କରାଯାଇ ▁ନଥିଲା । ▁ତାଙ୍କ ▁ପେଟ ର ▁ବାମ ▁ପାର୍ଶ୍ୱ ର ▁ମାଂସପେଶୀରେ
1,▁ନାଁ ▁ନୀ ରୁ ▁ଭେ ଡା ▁ବୋଲି ▁ଜଣା ▁ପଡ଼ିଛି ▁। ▁ପୂର୍ବ ବର୍ତ୍ତୀ ▁ଖବର ▁ଏହି ▁୪ଟି ▁ଲକ୍ଷଣ ▁ହୋଇପାରେ ▁ହୃଦ୍ ▁ଘାତ ର ▁କାରଣ ▁ପରବର୍ତ୍ତୀ ▁ଖବର ▁x x b os ▁0 ▁ସେ ୟାର ▁ଭୁବନେଶ୍ୱର : ▁ଯୁବକ ଙ୍କୁ ▁ଆକ୍ରମଣ ▁ଘଟଣା ରେ ▁ସହିଦ ନଗର ▁ଥାନା ▁ପୁଲିସ ▁ଦୁଇ ▁ଯୁ କ ଙ୍କୁ ▁ଗିରଫ ▁କରି ▁କୋର୍ଟ ▁ଚାଲାଣ ▁କରିଛି । ▁ସେମାନଙ୍କ ▁ନାଁ ▁ରମେଶ ▁ଗୁଣ ି ▁ <unk> ୩୬ ) ▁ଓ ▁ପପୁ ▁ଗୁଣ ି ▁ <unk> ୨୨ ) । ▁ଘର ▁ସହିଦ
2,▁ସିଂହଙ୍କ ▁ମୁଣ୍ଡ କୁ ▁ଏକ ▁ଫର୍ ସା ରେ ▁ଚ ୋଟ ▁ପକାଇ ଥିଲା । ▁ପରେ ▁ଅଭିଯୁକ୍ତ ▁ପ୍ରଶାନ୍ତ ▁ନା ତ୍ ▁ତାଙ୍କ ▁ମୁଣ୍ଡ କୁ ▁ଲକ୍ଷ୍ୟ ▁କରି ▁ଗୁଳି ▁ଫୁଟାଇ ଥିଲା । ▁ଫଳରେ ▁ଘଟଣା ସ୍ଥଳ ରେ ▁ହିଁ ▁ଇନ୍ ▁ସ ପ ୍ଟେର ▁ଶ୍ରୀ ▁ସିଂହଙ୍କ ର ▁ମୃତ୍ୟୁ ▁ଘଟିଥିଲା । ▁ପୂର୍ବରୁ ▁ପୁଲ ଇ ସ ▁ପ୍ରଶାନ୍ତ ଙ୍କୁ ▁ଗିରଫ ▁କରି ▁ସାରି ▁ଥିବା ▁ବେଳେ ▁ଆଜି ▁କାଳ ୁଆ କୁ ▁ଆଜି ▁ଗିରଫ ▁କରାଯାଇଛି । ▁ଉଭୟ ଙ୍କୁ ▁ପୁଲିସ ▁ପଚରା ଉ ଚ ରା ▁କରୁଥିବା ▁ଜଣାଯାଇଛି ।
3,"▁ତୈଳ ▁ଦର ▁ଓ ▁ଡଲାର ▁ତୁଳନା ରେ ▁ଭାରତୀୟ ▁ଟଙ୍କାର ▁ମୂଲ୍ୟ ▁ବିଚାର କୁ ▁ନେଇ ▁ପେଟ ୍ରୋଲ ୍ ▁ଓ ▁ଡିଜ େ ଲ୍ ▁ ର ▁ଖୁ ଚୁରା ▁ଦର ▁ନିର୍ଧାର ଣ ▁କରାଯାଉଛି । ▁ପ୍ରତିଦିନ ▁ତେଲ ▁ଦର ▁ସଂଶୋଧନ ▁କରାଯାଉଛି । ▁ପୂର୍ବ ବର୍ତ୍ତୀ ▁ଖବର ▁x x b os <unk> st ▁up d ated <unk> ▁j ul ▁2 , ▁20 19 ▁p ho to - in dia to day ▁ସେ ୟାର ▁ମୁମ୍ବାଇ : ▁ରଣବୀର ▁ସିଂହଙ୍କ ▁ଅଭିନୀତ ▁ତଥା ▁କ"
4,"କୁ ▁କ ’ ଣ ▁ହୋଇଛି , ▁ଅବଶ୍ୟ ▁ଆମେ ▁ଜାଣି ନା ହୁଁ । ▁ସେଥିପାଇଁ ▁ଫିଲ୍ମ ର ▁ରିଲିଜ ୍ ▁ କୁ ▁ଅପେକ୍ଷା ▁କରାଯାଉ ! ▁‘ ଫୋ ର୍ ▁ଇ ଡି ୟ ଟ୍ ▁ସ୍ ▁ ’ ରେ ▁ସ ବ୍ୟ ▁ଓ ▁ଏଲିନା ଙ୍କର ▁ଗୋଟିଏ ▁ସୁନ୍ଦର ▁ଗୀତ ▁ଅଛି ▁‘‘ ଆ ଜି ଠାରୁ ▁ପାଦ ▁ମୋର ▁ତୋ ରି ▁ରାସ୍ତାରେ ... ’ ’ । ▁ଥାଇ ଲ୍ୟାଣ୍ଡ ୍ ▁ ର ▁ମନୋରମ ▁ଲୋକ େସନ୍ ▁ ରେ ▁ସୁଟିଂ ▁ହୋଇଥିବା ▁ଏ ▁ଗୀତଟି ▁ଏବେ"


In [22]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [24]:
# Loading the pretrained language model on odia wikipedia
learn.load('../../../models/oriya/lm/ULMFiT/third_or_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (24000 items)
x: LMTextList
▁x x b os <unk> ▁e di ted <unk> ▁b y <unk> r e port er <unk> ▁b ur ea u ▁| <unk> ▁b y <unk> ▁j <unk> ▁g ▁on ▁de c ▁23 , ▁20 18 <unk> ▁s ha re ▁ମେଲ ବର୍ଣ୍ଣ : ▁ଡିସେମ୍ବର ▁୨୬ ▁ତାରିଖ ରୁ ▁ଭାରତ ▁ଓ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ମଧ୍ୟରେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ମ୍ୟାଚ୍ ▁ଆରମ୍ଭ ▁ହେବ । ▁ଚାରିଟି ▁ମ୍ୟାଚ୍ ▁ବିଶିଷ୍ଟ ୍ ▁ଏହି ▁ସିରିଜ ୍ ▁ ରେ ▁ଏବେ ▁ଉଭୟ ▁ଦଳ ▁ବରା ବର ▁ସ୍ଥିତି ରେ ▁ରହିଛନ୍ତି । ▁ଭାରତ ▁ଆ ଡେ ଲେ ଡ୍ ▁ଟେଷ୍ଟ ୍ ▁ଜିତି ▁ଥିବା ▁ବେଳେ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ବାଜି ▁ମାରି ଥିଲା । ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ଭାରତ ▁ବିନା ▁ସ୍ପିନ ର ରେ ▁ପଡ଼ିଆ କୁ ▁ଓହ୍ଲାଇ ଥିଲା । ▁ଆହତ ▁ଥିବା ▁କାରଣରୁ ▁ପ୍ରଥମ ▁ଟେଷ୍ଟ ରେ ▁୬ଟି ▁ୱିକେଟ ୍ ▁ନେଇଥିବା ▁ରବି ଚନ୍ଦ୍ର ନ ▁ଅଶ୍ୱ ିନ୍ ▁ ଙ୍କୁ ▁ଏହି ▁ଟେଷ୍ଟ ରେ ▁ସାମିଲ ▁କରାଯାଇ ▁ନଥିଲା । ▁ତାଙ୍କ ▁ପେଟ ର ▁ବାମ ▁ପାର୍ଶ୍ୱ ର ▁ମାଂସପେଶୀରେ ▁ସମସ୍ୟା ▁ଦେଖାଯାଇଥିଲା । ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ସେ ▁ପୂରା ▁ଫିଟ ୍ ▁ହୋଇ ନାହାନ୍ତି । ▁ତେଣୁ ▁ସେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ଖେଳି ବେ କି ▁ନାହିଁ , ▁ତାହା ▁ଏଯାବତ ୍ ▁ସ୍ପଷ୍ଟ ▁ହୋଇନାହିଁ । ▁କୋଚ ୍ ▁ରବି ▁ଶାସ୍ତ୍ରୀ ଙ୍କ ▁ଠାରୁ ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ଆସ ନ୍ତା ▁୪୮ ▁ଘଣ୍ଟା ▁ଭ

In [25]:
learn.freeze()

In [26]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.234499,4.068701,0.326821,02:38


In [27]:
learn.unfreeze()

In [28]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.879493,3.813096,0.355793,03:11
1,3.617208,3.576561,0.383892,03:11
2,3.440516,3.457191,0.398898,03:11
3,3.322337,3.404660,0.406235,03:12
4,3.281008,3.396182,0.407210,03:12


In [29]:
learn.predict('ରେ ଅପ ଗ୍ ରେ ଡ',n_words=10)

'ରେ ଅପ ଗ୍ ରେ ଡ ଟ୍ ▁କ ଏନ୍ ▁ଏସ୍ ▁ , ▁ସେ ଜୁ ▁ନା ▁ଏୟାର'

In [30]:
learn.save_encoder('fine_tuned_enc')

In [31]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=oriya_vocab, label_cols=label_cols, bs=64)

In [32]:
data_clas.show_batch()

text,target
▁x x b os ▁0 ▁ସେ ୟାର ▁ଫୁଟ୍ ▁ବଲ୍ ▁ବିଶ୍ୱକପ ୍ ▁ ରେ ▁ବ୍ୟବହୃତ ▁ହୋଇ ▁ଆସୁଥିବା ▁ବଲ୍ ▁ ର ▁ରୂପ ▁ଓ ▁ଆକାରରେ ▁ନାଟକ ୀୟ ▁ପରିବର୍ତନ ▁ହୋଇଛି । ▁୧୯୩୦ ର ▁ପ୍ରଥମ ▁ବିଶ୍ୱକପ ୍ ▁ ରେ ▁ବ୍ୟବହୃତ ▁ହୋଇଥିବା ▁ବଲ୍ ▁ଏବେ ▁ପୂରା ▁ବଦଳି ▁ଯାଇଛି । ▁ବିଶ୍ୱକପ ୍ ▁ଇତିହାସ ▁କେତେକାଂଶ ରେ ▁ପେସା ଦାର ▁ଫୁଟ୍ ▁ବଲ୍ ▁ ର ▁ବିବର୍ତ୍ତନ ▁ବା ▁କ୍ରମ ବିକାଶ କୁ ▁ଦର୍ଶା ଉ ଥିଲା ▁ବେଳେ ▁ଏହାର ▁ପ୍ରଭାବ ▁ବଲ୍ ▁ଉପରେ ▁ମଧ୍ୟ ▁ପଡ଼ିଛି । ▁ଘୁ,sports
"▁x x b os <unk> st ▁up d ated <unk> ▁j ul ▁6 , ▁20 19 ▁ସେ ୟାର ▁ନୂଆଦିଲ୍ଲୀ : ▁ଏ ଣି କି ▁ପେଟ ୍ରୋଲ ▁ଓ ▁ଡିଜ େଲ ▁ଉପରେ ▁ଲିଟର ▁ପିଛା ▁୧ ▁ଟଙ୍କା ▁ସଡକ ▁ଓ ▁ଭିତ୍ତିଭୂମି ▁ସେ ସ୍ ▁ଓ ▁ଆଉ ▁୧ ଟଙ୍କା ▁ଅତିରିକ୍ତ ▁ଉତ୍ପାଦନ ▁ ଶୁଳ୍କ ▁ଏମିତି ▁ େ ମାଟ ▁୨ ▁ଟଙ୍କାର ▁ପି ଟ ୁଣୀ ▁ଟିକସ ▁ଦେବେ ▁ଉପଭୋକ୍ତା । ▁ସୁନା ▁ଓ ▁ଅନ୍ୟାନ୍ୟ ▁ମୂଲ୍ୟବାନ ▁ଧାତୁ ▁ଉପରେ ▁୧୦ % ▁ବଦଳରେ ▁୧୨ .",business
▁x x b os ▁0 ▁ସେ ୟାର ▁ୱେ ଲି ଂ ଟନ୍ : ▁ଭାରତୀୟ ▁କ୍ରିକେଟ ୍ ▁ଦଳ ▁ଗତ ▁ତିନି ▁ମାସ ▁ହେବ ▁ସଫଳତାର ▁ନୂଆ ▁ଫର୍ ଦ ▁ଯୋଡ଼ି ▁ଚାଲିଛି । ▁ଏହି ▁ଧାରା କୁ ▁ ଆହୁରି ▁ଆଗେଇ ▁ନେବା ▁ଲାଗି ▁ଟିମ୍ ▁ଇଣ୍ଡିଆ କୁ ▁ଆଉ ▁ଏକ ▁ଭଲ ▁ମଞ୍ଚ ▁ବି ▁ମିଳି ▁ଯାଇଛି । ▁ଦିନିକିଆ ▁ଶୃଙ୍ଖଳା ରେ ▁ନ୍ୟୁ ଜି ଲାଣ୍ଡ କୁ ▁୪ - ୧ ରେ ▁ଧରା ଶାୟୀ ▁କରିଥିବା ▁ଭାରତ ▁ସେଠାରେ ▁ତିନି ▁ଟିକି ଆ ▁ ଟ୍ ବେଣ୍ଟ ି -,sports
▁x x b os ▁ମ ଙ୍କ ଡି ଂ କୁ ▁ନେଇ ▁ମହାଭାରତ ▁ମ ଙ୍କ ଡି ଂ କୁ ▁ନେଇ ▁ମହାଭାରତ ▁ବି ନୁ ▁ମ ଙ୍କ ଡ ଙ୍କ ▁ନାଁ ରୁ ▁‘ ମ ଙ୍କ ଡି ଂ ’ ▁0 ▁ସେ ୟାର ▁ଜୟପୁର : ▁ରାଜସ୍ଥାନ ▁ରୟାଲ ୍ସ ▁ପ୍ରାରମ୍ଭିକ ▁ବ୍ୟାଟ୍ସମ୍ୟାନ ୍ ▁ଜୋ ସ୍ ▁ବଟ ଲର ଙ୍କୁ ▁କି ଙ୍ଗ୍ ସ ▁ଏକାଦଶ ▁ପଞ୍ଜାବ ▁ଅଧିନାୟକ ▁ରବି ଚନ୍ଦ୍ର ନ୍ ▁ଅ ଶ୍ ବି ନ୍ ▁ମ ଙ୍କ ଡି ଂ ▁କରିବା ▁ନେଇ ▁ବଡ଼ ▁ବିବାଦ ▁ମୁଣ୍ଡ,sports
"▁x x b os <unk> ▁e di ted <unk> ▁b y <unk> r e port er <unk> ▁b ur ea u ▁| <unk> ▁b y <unk> ▁r <unk> ▁j <unk> st ▁up d ated <unk> p ▁16 , ▁20 19 <unk> ▁s ha re ▁ଲଣ୍ଡନ : ▁ଜବ ର ଦ ସ୍ତ ▁ସିରିଜ ୍ । ▁ଆ ଶ େସ୍ ▁ସିରିଜ ରେ ▁କ୍ରିକେଟ ର ▁ରୋମାଞ୍ଚ ▁ଶିଖର ରେ ▁ପହଞ୍ଚିଥିଲା । ▁ଚିର ▁ପ୍ରତି ଦ୍ୱନ୍ଦ ୍ୱ ୀ",sports


In [33]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [34]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [35]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (24000 items)
x: TextList
▁x x b os <unk> ▁e di ted <unk> ▁b y <unk> r e port er <unk> ▁b ur ea u ▁| <unk> ▁b y <unk> ▁j <unk> ▁g ▁on ▁de c ▁23 , ▁20 18 <unk> ▁s ha re ▁ମେଲ ବର୍ଣ୍ଣ : ▁ଡିସେମ୍ବର ▁୨୬ ▁ତାରିଖ ରୁ ▁ଭାରତ ▁ଓ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ମଧ୍ୟରେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ମ୍ୟାଚ୍ ▁ଆରମ୍ଭ ▁ହେବ । ▁ଚାରିଟି ▁ମ୍ୟାଚ୍ ▁ବିଶିଷ୍ଟ ୍ ▁ଏହି ▁ସିରିଜ ୍ ▁ ରେ ▁ଏବେ ▁ଉଭୟ ▁ଦଳ ▁ବରା ବର ▁ସ୍ଥିତି ରେ ▁ରହିଛନ୍ତି । ▁ଭାରତ ▁ଆ ଡେ ଲେ ଡ୍ ▁ଟେଷ୍ଟ ୍ ▁ଜିତି ▁ଥିବା ▁ବେଳେ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ବାଜି ▁ମାରି ଥିଲା । ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ଭାରତ ▁ବିନା ▁ସ୍ପିନ ର ରେ ▁ପଡ଼ିଆ କୁ ▁ଓହ୍ଲାଇ ଥିଲା । ▁ଆହତ ▁ଥିବା ▁କାରଣରୁ ▁ପ୍ରଥମ ▁ଟେଷ୍ଟ ରେ ▁୬ଟି ▁ୱିକେଟ ୍ ▁ନେଇଥିବା ▁ରବି ଚନ୍ଦ୍ର ନ ▁ଅଶ୍ୱ ିନ୍ ▁ ଙ୍କୁ ▁ଏହି ▁ଟେଷ୍ଟ ରେ ▁ସାମିଲ ▁କରାଯାଇ ▁ନଥିଲା । ▁ତାଙ୍କ ▁ପେଟ ର ▁ବାମ ▁ପାର୍ଶ୍ୱ ର ▁ମାଂସପେଶୀରେ ▁ସମସ୍ୟା ▁ଦେଖାଯାଇଥିଲା । ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ସେ ▁ପୂରା ▁ଫିଟ ୍ ▁ହୋଇ ନାହାନ୍ତି । ▁ତେଣୁ ▁ସେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ଖେଳି ବେ କି ▁ନାହିଁ , ▁ତାହା ▁ଏଯାବତ ୍ ▁ସ୍ପଷ୍ଟ ▁ହୋଇନାହିଁ । ▁କୋଚ ୍ ▁ରବି ▁ଶାସ୍ତ୍ରୀ ଙ୍କ ▁ଠାରୁ ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ଆସ ନ୍ତା ▁୪୮ ▁ଘଣ୍ଟା ▁ଭିତରେ 

In [36]:
learn.freeze()

In [37]:
learn.loss_func.func

CrossEntropyLoss()

In [38]:
mcc = MatthewsCorreff()

In [39]:
learn.metrics = [mcc, accuracy]

In [40]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.090517,0.060342,0.976005,0.982000,01:10


In [41]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.067633,0.045970,0.983556,0.987667,01:22


In [42]:
learn.save('second-full')

In [43]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.039403,0.048107,0.981330,0.986000,02:50
1,0.031007,0.057421,0.980884,0.985667,02:49
2,0.009417,0.058053,0.981770,0.986333,02:49
3,0.005693,0.060677,0.981772,0.986333,02:49
4,0.002998,0.059616,0.983993,0.988000,02:49


Better model found at epoch 0 with accuracy value: 0.9860000014305115.
Better model found at epoch 2 with accuracy value: 0.9863333106040955.
Better model found at epoch 4 with accuracy value: 0.9879999756813049.


In [44]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (24000 items)
x: TextList
▁x x b os <unk> ▁e di ted <unk> ▁b y <unk> r e port er <unk> ▁b ur ea u ▁| <unk> ▁b y <unk> ▁j <unk> ▁g ▁on ▁de c ▁23 , ▁20 18 <unk> ▁s ha re ▁ମେଲ ବର୍ଣ୍ଣ : ▁ଡିସେମ୍ବର ▁୨୬ ▁ତାରିଖ ରୁ ▁ଭାରତ ▁ଓ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ମଧ୍ୟରେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ମ୍ୟାଚ୍ ▁ଆରମ୍ଭ ▁ହେବ । ▁ଚାରିଟି ▁ମ୍ୟାଚ୍ ▁ବିଶିଷ୍ଟ ୍ ▁ଏହି ▁ସିରିଜ ୍ ▁ ରେ ▁ଏବେ ▁ଉଭୟ ▁ଦଳ ▁ବରା ବର ▁ସ୍ଥିତି ରେ ▁ରହିଛନ୍ତି । ▁ଭାରତ ▁ଆ ଡେ ଲେ ଡ୍ ▁ଟେଷ୍ଟ ୍ ▁ଜିତି ▁ଥିବା ▁ବେଳେ ▁ଅଷ୍ଟ୍ରେଲିଆ ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ବାଜି ▁ମାରି ଥିଲା । ▁ପ ର୍ଥ ▁ଟେଷ୍ଟ ରେ ▁ଭାରତ ▁ବିନା ▁ସ୍ପିନ ର ରେ ▁ପଡ଼ିଆ କୁ ▁ଓହ୍ଲାଇ ଥିଲା । ▁ଆହତ ▁ଥିବା ▁କାରଣରୁ ▁ପ୍ରଥମ ▁ଟେଷ୍ଟ ରେ ▁୬ଟି ▁ୱିକେଟ ୍ ▁ନେଇଥିବା ▁ରବି ଚନ୍ଦ୍ର ନ ▁ଅଶ୍ୱ ିନ୍ ▁ ଙ୍କୁ ▁ଏହି ▁ଟେଷ୍ଟ ରେ ▁ସାମିଲ ▁କରାଯାଇ ▁ନଥିଲା । ▁ତାଙ୍କ ▁ପେଟ ର ▁ବାମ ▁ପାର୍ଶ୍ୱ ର ▁ମାଂସପେଶୀରେ ▁ସମସ୍ୟା ▁ଦେଖାଯାଇଥିଲା । ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ସେ ▁ପୂରା ▁ଫିଟ ୍ ▁ହୋଇ ନାହାନ୍ତି । ▁ତେଣୁ ▁ସେ ▁ତୃତୀୟ ▁ଟେଷ୍ଟ ୍ ▁ଖେଳି ବେ କି ▁ନାହିଁ , ▁ତାହା ▁ଏଯାବତ ୍ ▁ସ୍ପଷ୍ଟ ▁ହୋଇନାହିଁ । ▁କୋଚ ୍ ▁ରବି ▁ଶାସ୍ତ୍ରୀ ଙ୍କ ▁ଠାରୁ ▁ମିଳିଥିବା ▁ସୂଚନା ▁ଅନୁଯାୟୀ , ▁ଆସ ନ୍ତା ▁୪୮ ▁ଘଣ୍ଟା ▁ଭିତରେ 

In [45]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,business,sports,entertainment,crime
0,0\nସେୟାର\nନୂଆଦିଲ୍ଲୀ: ଗୁଡ୍‌ସ ଆଣ୍ଡ୍‌ ସର୍ଭିସ ଟ୍ୟା...,business,business,0.999998,3.43955e-07,3.69589e-07,1.68402e-06
1,ଫିଲ୍ମ ନିର୍ଦେଶକଙ୍କୁ ଖଣ୍ଡାରେ ଆକ୍ରମଣ\nଫିଲ୍ମ ନିର୍ଦ...,crime,crime,2.44781e-08,3.4589e-07,7.66491e-05,0.999923
2,ତୈମୁରଙ୍କ ପିତା ହେବାକୁ ଚାହାନ୍ତି ଏହି ଅଭିନେତା !\nତ...,entertainment,entertainment,9.1453e-06,1.74593e-08,0.999988,2.62019e-06
3,Edited By OdishaReporter Bureau | By R M\nOn M...,sports,sports,7.36266e-08,0.999967,5.55237e-07,3.1885e-05
4,Edited By OdishaReporter Bureau | By R M\nOn N...,entertainment,entertainment,5.96005e-08,4.73664e-08,1,7.78201e-08


In [46]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9883333333333333

In [47]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.984445711337869

In [48]:
df_result.to_csv('indicnlp_news_articles_or_result.csv', index=False)